# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
dfg=data.groupby(["CustomerID", "ProductName"]).sum()

In [86]:
display(dfg)

SalesID  ProductID  Quantity
CustomerID ProductName                                                
33         Apricots - Dried                172592        324         1
           Assorted Desserts              5363705        219         1
           Bandage - Flexible Neon        6299165        273         1
           Bar Mix - Pina Colada, 355 Ml  3147536        281         1
           Beans - Kidney, Canned         6507033        300         1
...                                           ...        ...       ...
98200      Vol Au Vents                   5160346        272        50
           Wasabi Powder                  4438427         80        25
           Wine - Fume Blanc Fetzer       6226060         76        25
           Wine - Hardys Bankside Shiraz   266676        360        25
           Yogurt - French Vanilla        2112061        320        25

[63628 rows x 3 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

 ----------------
 pandas.pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, dropna=True, margins_name='All', observed=False)
 -------------------

In [87]:
pivot=pd.pivot_table(dfg, index="ProductName", values="Quantity", columns="CustomerID")

In [88]:
pivot_not_nan=pivot.fillna(0)

In [89]:
pivot_not_nan[33]

ProductName
Anchovy Paste - 56 G Tube            0.0
Appetizer - Mini Egg Roll, Shrimp    0.0
Appetizer - Mushroom Tart            0.0
Appetizer - Sausage Rolls            0.0
Apricots - Dried                     1.0
                                    ... 
Yeast Dry - Fermipan                 0.0
Yoghurt Tubes                        0.0
Yogurt - Blueberry, 175 Gr           0.0
Yogurt - French Vanilla              1.0
Zucchini - Yellow                    0.0
Name: 33, Length: 452, dtype: float64

In [90]:
pivot_not_nan

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

-------------

#1st way
- calculate the distance between users
- insert that distance in squareform

---------------

In [91]:
distances = pdist(pivot_not_nan, 'euclidean')
print(distances)
print("-----Squareform-----")
print(squareform(distances))

[279.99642855 253.66907577 238.5120542  ... 264.14200726 268.28343221
 273.15014186]
-----Squareform-----
[[  0.         279.99642855 253.66907577 ... 255.67166444 258.0910692
  278.47800631]
 [279.99642855   0.         271.49953959 ... 265.7367118  279.55500353
  286.80655502]
 [253.66907577 271.49953959   0.         ... 245.27535547 253.01580978
  269.43644891]
 ...
 [255.67166444 265.7367118  245.27535547 ...   0.         264.14200726
  268.28343221]
 [258.0910692  279.55500353 253.01580978 ... 264.14200726   0.
  273.15014186]
 [278.47800631 286.80655502 269.43644891 ... 268.28343221 273.15014186
    0.        ]]


---------------

# 2nd way: the chachi pistachi dataframe normalized
# IMPORTANT
- the same than before
    - with column names
    - with cool frame
    - beware the normalization
        - +3 to charisma
        - -4 to armor
    
------------

In [92]:
distances = pd.DataFrame(1/(1 + squareform(pdist(pivot_not_nan.T, 'euclidean'))), 
                         index=pivot_not_nan.columns, columns=pivot_not_nan.columns)

distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [93]:
# I choose the number 33 customer, and I'll check the closest customers affinity
distances.sort_values(by=[33],ascending=False).head(6)

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
3317,0.087047,0.084959,0.084297,0.085638,0.085638,0.086688,0.081210,0.088530,0.085297,0.083651,...,0.004806,0.004685,0.004415,0.005039,0.004309,0.004525,0.004579,0.004360,0.004164,0.004332
3535,0.087047,0.082403,0.086333,0.082403,0.084959,0.085983,0.083651,0.087779,0.087410,0.084959,...,0.004817,0.004677,0.004415,0.005036,0.004317,0.004522,0.004582,0.004364,0.004169,0.004348
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
2503,0.085983,0.080350,0.082709,0.083333,0.080350,0.085638,0.077171,0.085983,0.085638,0.081503,...,0.004814,0.004670,0.004410,0.005020,0.004303,0.004534,0.004581,0.004355,0.004173,0.004324
3305,0.085638,0.077421,0.084297,0.084297,0.080070,0.082709,0.077421,0.086333,0.082100,0.081800,...,0.004814,0.004672,0.004419,0.005029,0.004308,0.004527,0.004576,0.004359,0.004171,0.004336


In [94]:
distances[33].sort_values()

CustomerID
92255    0.004007
97324    0.004032
95017    0.004077
92492    0.004121
91777    0.004140
           ...   
2503     0.085983
3317     0.087047
3535     0.087047
264      0.087047
33       1.000000
Name: 33, Length: 1000, dtype: float64

In [95]:
# there they are. Column 0, elements [1:5]
ids=[]
for e in distances[33].sort_values(ascending=False)[1:6].index:
    ids.append(e)
ids

'''
Cómo hacerlo de una manera que no me hace ninguna ilusión
lst = []
list(map(lambda x: lst.append(int(x)), list((customer[1:6].index))))
'''

'\nCómo hacerlo de una manera que no me hace ninguna ilusión\nlst = []\nlist(map(lambda x: lst.append(int(x)), list((customer[1:6].index))))\n'

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [96]:
df5=data.groupby(["CustomerID", "ProductName"]).sum()
display(df5)

SalesID  ProductID  Quantity
CustomerID ProductName                                                
33         Apricots - Dried                172592        324         1
           Assorted Desserts              5363705        219         1
           Bandage - Flexible Neon        6299165        273         1
           Bar Mix - Pina Colada, 355 Ml  3147536        281         1
           Beans - Kidney, Canned         6507033        300         1
...                                           ...        ...       ...
98200      Vol Au Vents                   5160346        272        50
           Wasabi Powder                  4438427         80        25
           Wine - Fume Blanc Fetzer       6226060         76        25
           Wine - Hardys Bankside Shiraz   266676        360        25
           Yogurt - French Vanilla        2112061        320        25

[63628 rows x 3 columns]

In [97]:

sim_customers =df5.loc[ids]
sim_customers

SalesID  ProductID  Quantity
CustomerID ProductName                                                 
264        Apricots - Halves               3730301        278         1
           Apricots Fresh                  5035449        158         1
           Bacardi Breezer - Tropical       756802         49         1
           Bagel - Plain                   6728381        262         1
           Banana - Leaves                 6298459        271         1
...                                            ...        ...       ...
3535       Tray - 16in Rnd Blk             6122047        166         1
           Wanton Wrap                     2643670        253         1
           Wine - Blue Nun Qualitatswein   3314104        269         1
           Wine - Magnotta, Merlot Sr Vqa  6563480        192         1
           Yoghurt Tubes                   2723180        179         1

[270 rows x 3 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [98]:
data_aggregate= sim_customers.groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending = False)
data_aggregate

,Quantity
ProductName,
Butter - Unsalted,3
Wine - Ej Gallo Sierra Valley,3
Towels - Paper / Kraft,3
Soup - Campbells Bean Medley,3
Wine - Blue Nun Qualitatswein,3
...,...
Hinge W Undercut,1
Ice Cream Bar - Hageen Daz To,1
Jagermeister,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [99]:
data_not = data_aggregate.merge(pivot_not_nan[33], on='ProductName')
data_not

,Quantity,33
ProductName,,
Butter - Unsalted,3,0.0
Wine - Ej Gallo Sierra Valley,3,0.0
Towels - Paper / Kraft,3,1.0
Soup - Campbells Bean Medley,3,0.0
Wine - Blue Nun Qualitatswein,3,0.0
...,...,...
Hinge W Undercut,1,0.0
Ice Cream Bar - Hageen Daz To,1,0.0
Jagermeister,1,0.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [105]:
# Realmente me he quedado aquí. hoy no es mi día

# copiado de Clara
recommend = {}
customers = set(list(data['CustomerID']))
for cu in customers:
    simila = distances[cu].sort_values(ascending=False)[1:6]
    simi = list(simila.index)
    finali= dfg.loc[simi]
    #recom = finali.groupby(['ProductName'], as_index=False).agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending = False)
    recom = finali.reset_index().groupby(['ProductName']).agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending = False)
    recomm = recom.merge(pivot_not_nan[cu], on='ProductName')
    filtered = recomm.loc[recomm[cu]==0.0].sort_values(by='Quantity', ascending= False)[1:6]
    pasowhat =list(filtered.index)
    recommend[cu]=pasowhat



##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [106]:
df=pd.DataFrame(recommend)
df= df.T
df

,0,1,2,3,4
83973,Veal - Slab Bacon,Bandage - Fexible 1x3,Ezy Change Mophandle,Chicken - Wieners,Puree - Passion Fruit
59399,Towels - Paper / Kraft,Knife Plastic - White,Nantucket - Pomegranate Pear,Snapple - Iced Tea Peach,Cookie Chocolate Chip With
92168,Snapple Lemon Tea,"Veal - Brisket, Provimi,bnls",Spinach - Baby,Pop Shoppe Cream Soda,Bread - Calabrese Baguette
49159,Cookies - Assorted,Puree - Mocha,Cheese - Parmesan Grated,Cheese - Victor Et Berthold,"Chicken - Leg, Boneless"
18441,Beef - Texas Style Burger,Anchovy Paste - 56 G Tube,Flour - Pastry,Foam Cup 6 Oz,"Garlic - Primerba, Paste"
...,...,...,...,...,...
77814,Macaroons - Two Bite Choc,Tilapia - Fillets,Wine - Ruffino Chianti,Cake - Cake Sheet Macaroon,Guinea Fowl
32753,Sherry - Dry,"Chestnuts - Whole,canned",Napkin White - Starched,Sprouts - Alfalfa,Appetizer - Sausage Rolls
20476,"Pasta - Detalini, White, Fresh",Skirt - 29 Foot,Wanton Wrap,Broom - Corn,Ice Cream Bar - Hageen Daz To
67582,Ecolab - Solid Fusion,Wine - Redchard Merritt,Mussels - Frozen,Cheese - Bocconcini,"Peas - Pigeon, Dry"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.